# Basic characterizations of UA Q1
### using Experiment and Analysis classes

In [1]:
from qm.qua import *
from qm.QuantumMachinesManager import QuantumMachinesManager
from qm import SimulationConfig, LoopbackInterface
from qm.octave import *
from qm.octave.octave_manager import ClockMode
from configuration import *
from scipy import signal
from qm import SimulationConfig
from qualang_tools.bakery import baking
from qualang_tools.units import unit
from qm import generate_qua_script
from qm.octave import QmOctaveConfig
from set_octave import ElementsSettings, octave_settings
from quam import QuAM
from scipy.io import savemat
from scipy.io import loadmat
from scipy.optimize import curve_fit
from scipy.signal import savgol_filter
from qutip import *
from typing import Union
from macros import *
import datetime 
import os
import time
import warnings
import json
import matplotlib.pyplot as plt
import numpy as np
import Labber

from ExperimentClass import ExperimentHandle
from AnalysisClass import AnalysisHandle

warnings.filterwarnings("ignore")

qubit_index = 0 # for UA Q1
res_index = 0 # for UA Q1
flux_index = 0 # for UA Q1

2023-10-24 15:40:16,732 - qm - INFO     - Starting session: 17538def-e1e9-4556-b04b-bd3399b886bb


In [2]:
Experiment = ExperimentHandle()
Analysis = AnalysisHandle()
machine = Analysis.get_machine()

In [ ]:
# initialize Labber-controlled hardware
Experiment.set_Labber(Analysis.get_machine(), qubit_index, res_index, flux_index)
# set octave to external clock
#Experiment.set_external_clock()

In [ ]:
Experiment.octave_calibration(qubit_index,res_index,flux_index)

## Time of flight

In [ ]:
# time_of_flight
# experiment
machine, adc1, adc2, adc1_single, adc2_single = Experiment.exp1D.RR.time_of_flight(qubit_index,res_index,flux_index, n_avg = 10000, cd_time = 10E3, tPath = None, f_str_datetime = None, simulate_flag = False, simulation_len = 1000)
# plot
%matplotlib inline
dc_offset_i, dc_offset_q, delay = Analysis.exp1D.time_of_flight(adc1,adc2,adc1_single,adc2_single)

In [ ]:
# update the offset
machine.global_parameters.con1_downconversion_offset_I += 0.002220
machine.global_parameters.con1_downconversion_offset_Q += 0.002710
Analysis.set_machine(machine)

## 1D RO resonator spectroscopy

In [ ]:
%matplotlib qt
res_freq_sweep = np.arange(7.2575E9,7.2675E9 + 1.0,0.03E6)
machine, res_freq_sweep, sig_amp = Experiment.exp1D.RR.rr_freq(res_freq_sweep, qubit_index, res_index, flux_index, n_avg = 5E3, cd_time = 5E3, tPath = None, f_str_datetime = None, simulate_flag = False, simulation_len = 1000)

In [ ]:
# find the minimal
%matplotlib inline
res_freq = Analysis.exp1D.rr_freq(res_freq_sweep, sig_amp)
machine.resonators[res_index].f_readout = res_freq.tolist() + 0E6
machine = Analysis.set_machine(machine)

## 2D RO resonator spectroscopy vs DC flux

In [ ]:
res_freq_sweep = np.arange(7.254E9,7.267E9 + 1.0,0.1E6)
dc_flux_sweep = np.arange(-9,9.001,0.5)
%matplotlib qt
machine, res_freq_sweep, dc_flux_sweep, sig_amp = Experiment.exp2D.RR.rr_vs_dc_flux(res_freq_sweep, dc_flux_sweep, qubit_index, res_index, flux_index, n_avg = 1E3, cd_time = 10E3)

In [ ]:
%matplotlib inline
ham_param = Analysis.exp2D.rr_vs_dc_flux(res_freq_sweep, dc_flux_sweep, sig_amp, init_guess = None)
poly_param = Analysis.exp2D.qubit_vs_dc_flux_guess(ham_param)

In [ ]:
# save the fitting parameters
machine.resonators[res_index].tuning_curve = ham_param.tolist()
machine.qubits[qubit_index].tuning_curve = poly_param.tolist()
machine = Analysis.set_machine(machine)

In [ ]:
# save the sweep spot flux according to fitted parameter
machine.flux_lines[flux_index].max_frequency_point = Analysis.get_sweept_spot(machine.qubits[qubit_index].tuning_curve)
machine = Analysis.set_machine(machine)

## 1D qubit spectroscopy @ sweep spot

In [ ]:
# set to sweet spot
machine = Experiment.set_QDAC(qubit_index,res_index,flux_index,machine.flux_lines[flux_index].max_frequency_point)

In [ ]:
# calibrate RO
%matplotlib qt
#res_freq_sweep = np.arange(7.2575E9,7.2675E9 + 1.0,0.03E6)
ham_param = machine.resonators[res_index].tuning_curve
res_freq_est = Analysis.exp2D.ham([flux_index,machine.flux_lines[flux_index].max_frequency_point], ham_param[0], ham_param[1], ham_param[2], ham_param[3], ham_param[4], ham_param[5], output_flag = 1) * 1E6 # to Hz
res_freq_sweep = int(res_freq_est[0]) + np.arange(-5E6, 5E6 + 1, 0.05E6)
machine, res_freq_sweep, sig_amp = Experiment.exp1D.RR.rr_freq(res_freq_sweep, qubit_index, res_index, flux_index, n_avg = 5E3, cd_time = 5E3, plot_flag = False, tPath = None, f_str_datetime = None, simulate_flag = False, simulation_len = 1000)
%matplotlib inline
res_freq = Analysis.exp1D.rr_freq(res_freq_sweep, sig_amp)
res_freq = int(res_freq.tolist())
machine.resonators[res_index].f_readout = res_freq + 0E6
machine.resonators[res_index].lo = res_freq + 50E6
machine = Analysis.set_machine(machine)

In [ ]:
# calibrate octave
Experiment.octave_calibration(qubit_index,res_index,flux_index)

In [ ]:
#qubit_freq_sweep = np.arange(6.30E9,6.60E9+1,1E6)
qubit_freq_est = np.polyval(machine.qubits[qubit_index].tuning_curve,machine.flux_lines[flux_index].max_frequency_point) * 1E6 # in Hz
qubit_freq_sweep = int(qubit_freq_est) + np.arange(-50E6, 50E6 + 1, 1E6)
    
%matplotlib qt
machine, qubit_freq_sweep, sig_amp = Experiment.exp1D.Rabi.qubit_freq(qubit_freq_sweep, qubit_index, res_index, flux_index, 
                                      pi_amp_rel = 1.0, ff_amp = 0.0, n_avg = 2E3, cd_time = 10E3, plot_flag = False)

In [ ]:
%matplotlib inline
qubit_freq = Analysis.exp1D.peak_fit(x=qubit_freq_sweep,y=sig_amp,method="gaussian")
qubit_freq = int(qubit_freq.tolist())
machine.qubits[qubit_index].f_01 = qubit_freq + 0E6
machine.qubits[qubit_index].lo = qubit_freq + 100E6
machine = Analysis.set_machine(machine)

In [ ]:
# calibrate octave
Experiment.octave_calibration(qubit_index,res_index,flux_index)

## Rabi

### time rabi

In [ ]:
%matplotlib qt
rabi_duration_sweep = np.arange(4, 40 + 0.1, 1)  # in units of clock cyles
rabi_duration_sweep = rabi_duration_sweep.astype(int)
machine, rabi_duration_sweep, sig_amp = Experiment.exp1D.Rabi.rabi_length(rabi_duration_sweep, qubit_index, res_index, flux_index, 
                                          pi_amp_rel = 1.0, n_avg = 2E3, cd_time = 10E3, plot_flag = False)

In [ ]:
%matplotlib inline
pi_pulse_length = Analysis.exp1D.rabi_length(rabi_duration_sweep,sig_amp,plot_flag = True)

In [ ]:
machine.qubits[qubit_index].pi_length[0] = 36
machine = Analysis.set_machine(machine)

### power rabi

In [ ]:
%matplotlib qt
rabi_amp_sweep_rel = np.arange(0.3, 2.0, 0.05) 
machine, rabi_amp_sweep_abs, sig_amp = Experiment.exp1D.Rabi.rabi_amp(rabi_amp_sweep_rel, qubit_index, res_index, flux_index, 
                                          n_avg = 2E3, cd_time = 10E3, plot_flag = False)

In [ ]:
%matplotlib inline
pi_pulse_amp = Analysis.exp1D.rabi_length(rabi_amp_sweep_abs,sig_amp,method = "power_rabi", plot_flag = True)

In [ ]:
machine.qubits[qubit_index].pi_amp[0] = pi_pulse_amp.tolist()
machine = Analysis.set_machine(machine)

## 2D qubit spectroscopy vs dc flux

In [ ]:
# try the experiment
%matplotlib qt
dc_flux_sweep = machine.flux_lines[flux_index].max_frequency_point + np.arange(-0.6,0.6+1E-4,0.05)
_,qubit_freq_sweep,dc_flux_sweep,sig_amp_qubit = Experiment.exp2D.Rabi.qubit_freq_vs_dc_flux(dc_flux_sweep,qubit_index,res_index,flux_index,
                                 n_avg = 2E3, cd_time = 10E3, pi_amp_rel = 1.0, plot_flag = True)

In [ ]:
# udpate the sweet spot
%matplotlib inline
poly_param = Analysis.exp2D.qubit_vs_dc_flux(qubit_freq_sweep, dc_flux_sweep, sig_amp_qubit, fit_order = 2, plot_flag = False)
machine.flux_lines[flux_index].max_frequency_point = Analysis.get_sweept_spot(poly_param).tolist() + 0E1
print(f"sweet spot flux: {machine.flux_lines[flux_index].max_frequency_point:.5f} V")

In [ ]:
# save the sweep spot flux according to fitted parameter
poly_param = Analysis.exp2D.qubit_vs_dc_flux(qubit_freq_sweep, dc_flux_sweep, sig_amp_qubit, fit_order = 4, plot_flag = True)
machine.qubits[qubit_index].tuning_curve = poly_param.tolist()
machine = Analysis.set_machine(machine)
Analysis.update_analysis_tuning_curve(qubit_index,res_index,flux_index)

## 2D qubit spectroscopy vs fast flux

In [3]:
# set DC voltage to sweet spot, after possible dc flux sweep
machine = Experiment.set_QDAC(qubit_index,res_index,flux_index,machine.flux_lines[flux_index].max_frequency_point)
machine = Analysis.set_machine(machine)

### slow version, compiling 1D scans

In [ ]:
# try the experiment
%matplotlib qt
ff_sweep = np.arange(0.0,0.3+1E-4,0.05)
qubit_if_sweep = np.arange(-100E6, 100E6 + 1, 2E6)
machine,qubit_freq_sweep,ff_sweep_abs,sig_amp_qubit = Experiment.exp2D.Rabi.qubit_freq_vs_fast_flux_slow(ff_sweep,qubit_if_sweep,
          qubit_index,res_index,flux_index,ff_to_dc_ratio = None, n_avg = 1E3, cd_time = 10E3, pi_amp_rel = 1.0, plot_flag = True)

In [ ]:
# save the fitted tuning curve
%matplotlib inline
poly_param = Analysis.exp2D.qubit_vs_fast_flux(qubit_freq_sweep, ff_sweep_abs, sig_amp_qubit, fit_order = 4, plot_flag = True)
machine.qubits[qubit_index].tuning_curve = poly_param.tolist()
machine = Analysis.set_machine(machine)
Analysis.update_analysis_tuning_curve(qubit_index,res_index,flux_index)

### fast version, runs blockwise 2D sweeps

In [4]:
# try the experiment
%matplotlib qt
ff_sweep_abs = np.arange(0.0,0.3+1E-4,0.05)
qubit_if_sweep = np.arange(-100E6, 100E6 + 1, 2E6)
machine,qubit_freq_sweep,ff_sweep_abs,sig_amp_qubit = Experiment.exp2D.Rabi.qubit_freq_vs_fast_flux(ff_sweep_abs,qubit_if_sweep,
          qubit_index,res_index,flux_index,ff_to_dc_ratio = None, n_avg = 1E3, cd_time = 10E3, pi_amp_rel = 1.0, plot_flag = True)

[6.42653935e+09 6.40973340e+09 6.35913009e+09 6.27417302e+09
 6.15393482e+09 5.99711721e+09]
2023-10-24 15:40:35,542 - qm - INFO     - Octave "octave1" Health check passed, current temperature 58


INFO:qm:Octave "octave1" Health check passed, current temperature 58


2023-10-24 15:40:35,545 - qm - INFO     - Performing health check


INFO:qm.api.frontend_api:Performing health check


2023-10-24 15:40:35,564 - qm - INFO     - Health check passed


...

2023-10-24 15:41:07,607 - qm - ERROR    - Internal error: java.lang.IllegalStateException: given number cannot be presented by QUA fixed representation
at io.qualang.qua.ProgramConverter.toQuaFixed(ProgramConverter.kt:34)
at io.qualang.qua.ProgramConverter.convert(ProgramConverter.kt:67)
at io.qualang.qua.ProgramConverter.convert(ProgramConverter.kt:10)
at qm.opx.gateway.CompileKt.rawCompile(compile.kt:62)
at qm.opx.gateway.CompileKt.compile(compile.kt:82)
at qm.opx.gateway.GatewayService$addToQueue$2$1.invoke(GatewayService.kt:152)
at qm.opx.gateway.GatewayService$addToQueue$2$1.invoke(GatewayService.kt:151)
at qm.opx.gateway.ThreadWithBlockingTimeoutKt.run$lambda$0(ThreadWithBlockingTimeout.kt:27)
at java.base/java.lang.Thread.run(Unknown Source)


ERROR:qm.api.frontend_api:Internal error: java.lang.IllegalStateException: given number cannot be presented by QUA fixed representation
at io.qualang.qua.ProgramConverter.toQuaFixed(ProgramConverter.kt:34)
at io.qualang.qua.ProgramConverter.convert(ProgramConverter.kt:67)
at io.qualang.qua.ProgramConverter.convert(ProgramConverter.kt:10)
at qm.opx.gateway.CompileKt.rawCompile(compile.kt:62)
at qm.opx.gateway.CompileKt.compile(compile.kt:82)
at qm.opx.gateway.GatewayService$addToQueue$2$1.invoke(GatewayService.kt:152)
at qm.opx.gateway.GatewayService$addToQueue$2$1.invoke(GatewayService.kt:151)
at qm.opx.gateway.ThreadWithBlockingTimeoutKt.run$lambda$0(ThreadWithBlockingTimeout.kt:27)
at java.base/java.lang.Thread.run(Unknown Source)


2023-10-24 15:41:07,609 - qm - ERROR    - Job 1696378113517 failed. Failed to execute program.


ERROR:qm.api.frontend_api:Job 1696378113517 failed. Failed to execute program.


FailedToAddJobToQueueException: Job 1696378113517 failed. Failed to execute program.

### test the code

In [ ]:
ff_to_dc_ratio = None

In [ ]:
poly_param = machine.qubits[qubit_index].tuning_curve
ff_sweep_abs = np.arange(0.0,0.3+1E-4,0.05)

In [ ]:
ff_sweep = ff_sweep_abs / machine.flux_lines[flux_index].flux_pulse_amp
if ff_to_dc_ratio is None:
    qubit_freq_est_sweep = np.polyval(poly_param, ff_sweep_abs) * 1E6 # Hz
else:
    qubit_freq_est_sweep = np.polyval(poly_param, (ff_to_dc_ratio * ff_sweep_abs) + machine.flux_lines[flux_index].max_frequency_point) * 1E6 # Hz
qubit_freq_est_sweep = np.round(qubit_freq_est_sweep)

In [ ]:
qubit_freq_est_sweep

In [ ]:
# divide and conquer!
freq_est_seg_index = [0]
for freq_est_index, freq_est_value in enumerate(qubit_freq_est_sweep):
    if freq_est_value < qubit_freq_est_sweep[freq_est_seg_index[-1]] - 500E6:
        freq_est_seg_index.append(freq_est_index)
freq_est_seg_index.append(len(qubit_freq_est_sweep))

In [ ]:
freq_est_seg_index

In [ ]:
for freq_seg_index, freq_est_value in enumerate(freq_est_seg_index[1:2]):
    index_seg_lower = freq_est_seg_index[freq_seg_index]
    index_seg_upper = freq_est_value

    ff_sweep_rel_seg = ff_sweep[index_seg_lower:index_seg_upper]
    qubit_freq_est_sweep_seg = qubit_freq_est_sweep[index_seg_lower:index_seg_upper]
    print(qubit_freq_est_sweep_seg)
    qubit_lo = max(qubit_freq_est_sweep_seg) + max(qubit_if_sweep) - 350E6
    machine.qubits[qubit_index].lo = int(qubit_lo.tolist()) + 0E6
    machine.qubits[qubit_index].f_01 = int(max(qubit_freq_est_sweep_seg).tolist()) + 0E6
    Experiment.octave_calibration(qubit_index,res_index,flux_index,machine = machine)

In [ ]:
qubit_freq_est_sweep_seg

In [ ]:
qubit_lo = machine.qubits[qubit_index].lo

In [ ]:
qubit_if_est_sweep = np.round(qubit_freq_est_sweep_seg - qubit_lo)

In [ ]:
qubit_freq_sweep_tot = []
for qubit_freq_i in qubit_freq_est_sweep:
    qubit_freq_sweep_tot.append(qubit_freq_i + qubit_if_sweep)

In [ ]:
np.size(qubit_freq_sweep_tot)

In [ ]:
ff_sweep_rel_seg

In [ ]:
qubit_if_est_sweep

In [ ]:
qubit_if_sweep

In [ ]:
machine, qubit_freq_sweep_tmp, I_tmp, Q_tmp, ff_sweep_tmp = Experiment.exp2D.Rabi.qubit_freq_fast_flux_subroutine(
        ff_sweep_rel_seg, qubit_freq_est_sweep_seg, qubit_if_sweep,
        qubit_index, res_index, flux_index, pi_amp_rel=1.0, n_avg=100, cd_time=10E3, machine=machine)